# Модель

In [ ]:
from InceptionTime import Latent, InceptionBlock
import torch
import torch.nn as nn


class TimeSeriesClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()

        self.encoder = InceptionBlock(
            in_channels=1,
            n_filters=8,
            kernel_sizes=[9, 19, 39],
            bottleneck_channels=32,
            use_residual=True,
            activation=nn.ReLU(),
            return_indices=True
        )
        
        self.latent = Latent()

        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x shape: (batch_size, sequence_len)
        x = x.unsqueeze(1)  # x shape: (batch_size, 1, sequence_len)
        x, indices = self.encoder(x)
        x, _ = self.latent(x, indices)
        x = x.squeeze(dim=3)
        x = self.classifier(x)
        return x


In [ ]:
model_classifier=TimeSeriesClassifier(input_size=1, hidden_size=32, num_layers=2)

In [ ]:
for param in model_classifier.encoder.parameters():
            param.requires_grad = True
for param in model_classifier.classifier.parameters():
            param.requires_grad = False

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

def accuracy(outputs, targets):
    with torch.no_grad():
        predicted = outputs
        correct = predicted.eq(targets).sum().item()
        accuracy = correct / targets.numel()
    return accuracy

def precision(outputs, targets):
    with torch.no_grad():
        predicted = outputs
        true_positives = (predicted * targets).sum().item()
        false_positives = ((predicted == 1) & (targets == 0)).sum().item()
        precision = true_positives / (true_positives + false_positives)
    return precision

# Создание пустого списка для потерь
loss_list=[]

# Определение устройства для вычислений
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Перемещаем модель на вычислительное устройство
model_classifier.to(device)

# Выводим использованное вычислительное устройство
print(device)

# Определение количества эпох для обучения\nnum_epochs = 30

# Определение функции потерь
criterion = nn.BCELoss()

# Определение оптимизатора и скорости обучения
optimizer = optim.Adam(model_classifier.parameters(), lr=0.00005)

# Инициализация переменной для лучшей потери
best_loss = float('inf')

# Начало цикла обучения на num_epochs эпох
for epoch in range(num_epochs):

    # Инициализация переменных для подсчета потерь и метрик
    running_loss = 0.0
    run_acc=0.0
    prec_acc=0.0

    # Переводим модель в режим обучения
    model_classifier.train()

    # Начало цикла по обучающим данным
    for data in tqdm(train_dataloader):

        # Перемещаем данные и метки на вычислительное устройство
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход: вычисляем выход модели для входных данных
        outputs = model_classifier(inputs)

        # Вычисляем функцию потерь
        loss = criterion(outputs, labels.unsqueeze(1))

        # Обратный проход и оптимизация параметров
        loss.backward()
        optimizer.step()

        # Суммируем потери по батчам
        running_loss += loss.item() * inputs.size(0)

    # Начало цикла по тестовым данным
    val_loss = 0
    model_classifier.eval()
    with torch.no_grad():
        test_loss = 0.0
        run_acc_test=0.0
        prec_acc_test=0.0
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Прямой проход: вычисляем выход модели для входных данных
            outputs = model_classifier(inputs)

            # Вычисляем функцию потерь
            loss = criterion(outputs, labels.unsqueeze(1))

            # Суммируем потери по батчам
            test_loss += loss.item() * inputs.size(0)
        
        # Считаем метрику accuracy
        run_acc_test += accuracy(outputs, labels)
        
        # Считаем метрику precision
        prec_acc_test += precision(outputs, labels)

    # Общая оценка потерь на тестовой выборке
    val_loss = _loss / len(test_loader.dataset)

    # Оценка точности на тестовой выборке
    accuracy_val = run_acc_test / len(test_loader)
    
    # Оценка precision на тестовой выборке
    precision_val = prec_acc_test / len(test_loader)

    # Обновляем переменную для лучшей потери
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model_classifier.state_dict(), 'best_model.pt')

    # Сохраняем потери для данной эпохи
    loss_list.append(val_loss)

    # Выводим результаты после завершения предыдущей эпохи
    print(f'Epoch: {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader.dataset)}, Validation Loss: {val_loss}, Validation Accuracy: {accuracy_val}, Precision: {precision_val}')



In [ ]:
#Visualize loss

import matplotlib.pyplot as plt
plt.plot(loss_list)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()